In [29]:
!pip -q install sagemaker sagemaker[local]
!pip -q install tensorboard
%load_ext tensorboard

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-data'
role = 'arn:aws:iam::294038372338:role/hunkimSagemaker'

In [4]:
%%bash
#wget http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
wget -O wikitext-2-raw-v1.zip https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
unzip -n wikitext-2-raw-v1.zip
cd wikitext-2-raw
mv wiki.test.raw test && mv wiki.train.raw train && mv wiki.valid.raw valid


Archive:  wikitext-2-raw-v1.zip
  inflating: wikitext-2-raw/wiki.test.raw  
  inflating: wikitext-2-raw/wiki.valid.raw  
  inflating: wikitext-2-raw/wiki.train.raw  
--2020-06-14 14:48:17--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.147.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.147.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip’

     0K .......... .......... .......... .......... ..........  1%  232K 20s
    50K .......... .......... .......... .......... ..........  2%  233K 19s
   100K .......... .......... .......... .......... ..........  3%  233K 19s
   150K .......... .......... .......... .......... ..........  4% 43.6M 14s
   200K .......... .......... .......... .......... ..........  5% 34.2M 11s
   250K .......... .......... .......... .......... ..........  6%  236K 12s


In [5]:
!head -5 wikitext-2-raw/train


 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series n

In [6]:
inputs = sagemaker_session.upload_data(path='wikitext-2-raw', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2


In [24]:
from sagemaker.debugger import TensorBoardOutputConfig
tb_suffix = "/tensorboard/data/emission"
tensorboard_output = inputs + tb_suffix

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=tensorboard_output,
    container_local_output_path=tb_suffix
)

F_CPP_MIN_LOG_LEVEL=3 AWS_REGION=us-east2 tensorboard --log-dir s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2/tensorboard/data/emission


In [25]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    source_dir='code',
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    #train_instance_type='local',
                    tensorboard_output_config=tensorboard_output_config,


                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    hyperparameters={
                        'epochs': 1,
                        'tied': True
                    })

In [26]:
estimator.fit({'training': inputs})

2020-06-14 07:56:05 Starting - Starting the training job...
2020-06-14 07:56:07 Starting - Launching requested ML instances......
2020-06-14 07:57:15 Starting - Preparing the instances for training......
2020-06-14 07:58:46 Downloading - Downloading input data
2020-06-14 07:58:46 Training - Downloading the training image............
2020-06-14 08:01:00 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-14 08:01:02,227 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-06-14 08:01:02,255 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-14 08:01:02,261 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-14 08:01:02,546 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-0

In [45]:
tensorboard_s3_output_path = estimator.latest_job_tensorboard_artifacts_path()
print(tensorboard_s3_output_path)
%tensorboard --logdir {tensorboard_output}

from tensorboard import notebook
notebook.list()

s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2/tensorboard/data/emission/pytorch-training-2020-06-14-07-56-02-352/tensorboard-output
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

Known TensorBoard instances:
  - port 6007: logdir s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2/tensorboard/data/emission (started 0:11:25 ago; pid 3777)
  - port 6009: logdir tensorboard-output (started 0:08:11 ago; pid 3897)
  - port 6006: logdir s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2/tensorboard/data/emission/pytorch-training-2020-06-14-07-56-02-352/tensorboard-output (started 2:58:37 ago; pid 89975)
  - port 6010: logdir ./tensorboard-output (started 0:07:40 ago; pid 3954)
  - port 6008: logdir tensorboard-output (started 0:08:36 ago; pid 3888)


In [10]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [11]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
print(trained_model_location)
model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='generate.py',
                     source_dir='code',
                     predictor_cls=JSONPredictor)

AttributeError: 'NoneType' object has no attribute 'name'

In [12]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Get the end point
endpoint = predictor.endpoint
print(endpoint)

NameError: name 'model' is not defined

In [13]:
#FIXME: This works, but the predictor using endpoint does not work
input = {
    'seed': 111,
    'temperature': 2.0,
    'words': 100
}
response = predictor.predict(input)
print(response)

NameError: name 'predictor' is not defined

In [14]:
sagemaker_session.delete_endpoint(predictor.endpoint)

NameError: name 'predictor' is not defined